In [151]:
from binance.client import Client
from binance.enums import HistoricalKlinesType
from datetime import datetime, timedelta
import pandas as pd
import pandas_ta as ta
pd.set_option('display.max_rows', 500)
import requests,json
import plotly.graph_objs as go

In [152]:
# Define column names for the DataFrame
colnames = [
    'time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time',
    'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
    'Taker buy quote asset volume', 'Ignore'
]

# Get all USDT pairs
BASE_URL = 'https://api.kucoin.com'
resp = requests.get(BASE_URL+'/api/v2/symbols')
ticker_list = json.loads(resp.content)
symbols = [ticker['symbol'] for ticker in ticker_list['data'] if str(ticker['symbol'][-4:]) == 'USDT']


interval = '1d'  # Supports intervals like 1m, 1H, 1D, etc.
start = (datetime.today() - timedelta(days=90)).strftime('%m/%d/%Y 00:00:00')   # Start date
end = datetime.today().strftime('%m/%d/%Y 00:00:00') # End date

client = Client()  # Initialize your Binance API client


In [153]:
dicionario_adx = {
    '0-25': 'Tendência fraca ou inexistente',
    '25-50': 'Tendência forte',
    '50-75': 'Tendência muito forte',
    '75-100': 'Tendência extremamente forte'
}

def check_adx(value):
    for key, name in dicionario_adx.items():
        range_start, range_end = map(int, key.split('-'))
        if range_start <= value <= range_end:
            return name
    return None

In [154]:
def getting_data(symbol):
    print('Getting ' + symbol)

    kucoin = requests.get(f'https://api.kucoin.com/api/v1/market/candles?type=1day&symbol={symbol}')
    if kucoin.status_code == 200:  # if result is GOOD
        data = json.loads(kucoin.text)  # load the json response from API
        df = pd.DataFrame(data['data'], columns=['unix', 'price_open', 'price_close', 'price_high', 'price_low', 'volume', 'turnover'])  # create dataframe out of json response
        df['symbol'] = f'{symbol.replace("-", "")}'
        df['time_period_start'] = pd.to_datetime(df['unix'].astype(int), unit='s')
        df.sort_values(by='time_period_start', inplace = True) 
        
  # add a human readable date
    else:
        print('Error on connection')
        pass



    df[['price_open', 'price_high', 'price_low', 'price_close']] = df[['price_open', 'price_high', 'price_low', 'price_close']].map(lambda x: float(x))

    # Calculate ADX, DI+ and DI- manually
    df[['adx', 'dmp', 'dmn']] = ta.adx(df['price_high'], df['price_low'], df['price_close'], length = 14)
    df['adx_trend'] = df['adx'].apply(check_adx)


    # # # Calculate ichimoku cloud  
    df[['leading_span_a', 'leading_span_b', 'conversion_line', 'base_line', 'lagging_span']] = ta.ichimoku(df['price_high'], df['price_low'], df['price_close'])[0]
    df[['macd', 'macd_hist', 'macd_signal']] = ta.macd(df['price_close']).map(lambda x: float(x))
    df['macd_delta'] = df['macd'] - df['macd_signal']

    return df


In [155]:
testing = []
for crypto in symbols[:50]:
    try:
        data = getting_data(crypto)
        if not data.empty:  # Check if the dataframe is not empty before appending
            testing.append(data)
    except:
        pass

if testing:  # Check if the list is not empty before concatenating
    df_concat = pd.concat(testing, ignore_index=True)
else:
    # Handle the case when no data is available
    df_concat = pd.DataFrame()

Getting AVA-USDT
Getting MTV-USDT
Getting KMD-USDT
Getting RFOX-USDT
Getting TEL-USDT
Getting TT-USDT
Getting AERGO-USDT
Getting XMR-USDT
Getting ATOM-USDT
Getting FTM-USDT
Getting VSYS-USDT
Getting CHR-USDT
Getting COTI-USDT
Getting BNB-USDT
Getting ALGO-USDT
Getting XEM-USDT
Getting XTZ-USDT
Getting ZEC-USDT
Getting ADA-USDT
Getting BOLT-USDT
Getting ARPA-USDT
Getting CHZ-USDT
Getting NOIA-USDT
Getting WIN-USDT
Getting BTT-USDT
Getting EOSC-USDT
Getting ONE-USDT
Getting TOKO-USDT
Getting LUNA-USDT
Getting SXP-USDT
Getting AKRO-USDT
Getting MAP-USDT
Getting AMPL-USDT
Getting DAG-USDT
Getting KPOL-USDT
Getting ARX-USDT
Getting NWC-USDT
Getting BEPRO-USDT
Getting VRA-USDT
Getting KSM-USDT
Getting DASH-USDT
Getting SUTER-USDT
Getting SENSO-USDT
Getting XDB-USDT
Getting SYLO-USDT
Getting DGB-USDT
Getting STX-USDT
Getting COMP-USDT
Getting CRO-USDT
Getting KAI-USDT


In [156]:
today = datetime.today().date()
df_adj = df_concat.loc[
                (df_concat['adx'] > 25) & 
                (df_concat['time_period_start'].dt.date == today) &

                # Ichimoku com menos preço maior do que linha de conversão e linha de base
                (df_concat['price_low'] > df_concat['conversion_line']) &
                (df_concat['price_low'] > df_concat['base_line']) &

                # MACD histograma maior do que 0 e signal maior do que macd
                # (df_concat['macd_hist'] > 0) &
                (df_concat['macd_delta'] > 0)
                ]
                  

In [157]:
df_adj[['symbol', 'time_period_start', 'price_open', 'volume', 'adx', 'adx_trend', 'macd_delta']].sort_values(by='adx', ascending=False).reset_index()

,index,symbol,time_period_start,price_open,volume,adx,adx_trend,macd_delta
0,2599,EOSCUSDT,2023-12-04,0.003110,3846966.4791,71.885754,Tendência muito forte,0.000127
1,4499,SYLOUSDT,2023-12-04,0.003538,4950060.5309,62.130346,Tendência muito forte,0.000120
2,4399,XDBUSDT,2023-12-04,0.000745,80572828.2126,52.438815,Tendência muito forte,0.000036
3,299,KMDUSDT,2023-12-04,0.270500,27832.5406,47.041388,Tendência forte,0.001167
4,3599,ARXUSDT,2023-12-04,0.002539,916913.1472,44.516912,Tendência forte,0.000064
5,2899,LUNAUSDT,2023-12-04,0.897700,5910393.32072294,44.210583,Tendência forte,0.023417
6,1099,VSYSUSDT,2023-12-04,0.001192,2131930.7694,43.851455,Tendência forte,0.000011
7,2299,NOIAUSDT,2023-12-04,0.037441,80478.7834,43.769806,Tendência forte,0.000311
8,3799,BEPROUSDT,2023-12-04,0.001016,20770897.9469,36.345444,Tendência forte,0.000008
9,99,AVAUSDT,2023-12-04,0.548229,1867.5294,36.040012,Tendência forte,0.001390


In [158]:
# df_concat.to_csv('crypto_data_kucoin.csv', index=False)

In [159]:
# # Create the candlestick trace for price
# candlestick = go.Candlestick(
#     x=df_concat['time_period_start'],
#     open=df_concat['price_open'],
#     high=df_concat['price_high'],
#     low=df_concat['price_low'],
#     close=df_concat['price_close'],
#     name='Price'
# )

# # Create the figure for price (candlestick chart)
# fig = go.Figure(data=candlestick)

# # Update layout and add title
# fig.update_layout(
#     title='Price Candlestick Chart using Plotly',
#     xaxis_title='Date',
#     yaxis_title='Price'
# )

# # Show the plot
# fig.show()

In [160]:
# # Create the ADX trace
# adx_line = go.Scatter(
#     x=df_concat['time_period_start'],
#     y=df_concat['adx'],
#     mode='lines',
#     name='ADX',
#     line=dict(color='black')  # Color for ADX line
# )

# # Create the DI+ trace
# plus_di_line = go.Scatter(
#     x=df_concat['time_period_start'],
#     y=df_concat['dmp'],
#     mode='lines',
#     name='DI+',
#     line=dict(color='green')  # Color for DI+
# )

# # Create the DI- trace
# minus_di_line = go.Scatter(
#     x=df_concat['time_period_start'],
#     y=df_concat['dmn'],
#     mode='lines',
#     name='DI-',
#     line=dict(color='red')  # Color for DI-
# )
# # Create a fixed line for value 20 in light grey
# fixed_line = go.Scatter(
#     x=df_concat['time_period_start'],
#     y=[20] * len(df_concat),  # Creating a list of 20s to match the length of the DataFrame
#     mode='lines',
#     name='Fixed Line (20)',
#     line=dict(color='lightgrey', dash='dash')  # Color and style for the fixed line
# )

# # Create the figure for ADX chart
# fig = go.Figure(data=[adx_line, plus_di_line, minus_di_line, fixed_line])

# # Update layout and add title
# fig.update_layout(
#     title='ADX, DI+ and DI- Line Chart with Fixed Line at 20 using Plotly',
#     xaxis_title='Date',
#     yaxis_title='Values'
# )

# # Show the plot
# fig.show()

In [161]:
# # Create Ichimoku Cloud chart
# fig2 = go.Figure()

# # Add candlestick chart
# fig2.add_trace(go.Candlestick(x=df_concat['time_period_start'],
#                              open=df_concat['price_open'],
#                              high=df_concat['price_high'],
#                              low=df_concat['price_low'],
#                              close=df_concat['price_close'],
#                              name='Candlestick'))

# # Add Ichimoku Cloud lines
# fig2.add_trace(go.Scatter(x=df_concat['time_period_start'], y=df_concat['conversion_line'], mode='lines', name='Conversion Line', line=dict(color='blue')))
# fig2.add_trace(go.Scatter(x=df_concat['time_period_start'], y=df_concat['base_line'], mode='lines', name='Base Line', line=dict(color='red')))
# fig2.add_trace(go.Scatter(x=df_concat['time_period_start'], y=df_concat['leading_span_a'], mode='lines', name='Leading Span A'))
# fig2.add_trace(go.Scatter(x=df_concat['time_period_start'], y=df_concat['leading_span_b'], mode='lines', name='Leading Span B'))

# # Update layout
# fig2.update_layout(title='Price Candlestick with Ichimoku Cloud Indicators',
#                   xaxis_title='Date',
#                   yaxis_title='Price',
#                   showlegend=True)

# # Display fig2ure
# fig2.show()